# Building models from Scratch

with PyTorch and fastai

## Setup

In [ ]:
from pathlib import Path
import torch
import numpy as np
import pandas as pd
from fastai.data.transforms import RandomSplitter
import torch.nn.functional as F

In [ ]:
ROOT_PATH = Path.cwd().parent
DATA_PATH = ROOT_PATH / "data" / "titanic"

## Load data

In [ ]:
train_df = pd.read_csv(DATA_PATH / "train.csv")

In [ ]:
train_df.info()

## Process data

In [ ]:
def preprocess(df: pd.DataFrame) -> pd.DataFrame:
    modes = df.mode().iloc[0]
    df.fillna(modes, inplace=True)
    df["Fare"] = np.log(df["Fare"] + 1)
    df = pd.get_dummies(df, columns=["Sex", "Pclass", "Embarked"])

    return df

In [ ]:
processed_df = preprocess(train_df)

In [ ]:
processed_df.info()

In [ ]:
target = "Survived"
features = [
    col
    for col in processed_df.columns
    if col not in ("PassengerId", "Survived", "Name", "Ticket", "Fare", "Cabin")
]

In [ ]:
t_target = torch.tensor(processed_df[target], dtype=torch.float)

In [ ]:
t_features = torch.tensor(
    processed_df[features].astype(np.float64).values, dtype=torch.float
)

## Modeling

### Linear model

In [ ]:
torch.manual_seed(1337)

In [ ]:
def calc_preds(coeffs: torch.Tensor, features: torch.Tensor) -> torch.Tensor:
    # return torch.sigmoid((coeffs*features).sum(axis=1))
    return torch.sigmoid(features @ coeffs)

In [ ]:
def calc_loss(
    coeffs: torch.Tensor, features: torch.Tensor, target: torch.Tensor
) -> torch.Tensor:
    return torch.abs(calc_preds(coeffs, features) - target).mean()

#### Get random coefficients for our features

In [ ]:
coeffs = torch.rand(t_features.shape[1]) - 0.5

In [ ]:
coeffs

#### Scale features

In [ ]:
vals, indices = t_features.max(dim=0)
t_features = t_features / vals

#### Create predictions from linear model by multiplying features with coefficents and adding them up

In [ ]:
preds = (t_features * coeffs).sum(axis=1)

In [ ]:
preds[:10]

#### Define loss function as average of absolute value between predictions and target

In [ ]:
loss = torch.abs(preds - t_target).mean()

In [ ]:
loss

#### Gradient descent step

In [ ]:
coeffs.requires_grad_()

In [ ]:
loss = calc_loss(coeffs, t_features, t_target)
loss.backward()
with torch.no_grad():
    coeffs.sub_(coeffs.grad * 0.1)
    coeffs.grad.zero_()
    print(calc_loss(coeffs, t_features, t_target))

#### Train the linear model

In [ ]:
def init_coeffs():
    # _coeffs = torch.rand(t_features.shape[1]) - 0.5
    _coeffs = torch.rand(t_features.shape[1], 1) * 0.1
    _coeffs.requires_grad_()
    return _coeffs

In [ ]:
def update_coeffs(coeffs: torch.Tensor, lr: float):
    coeffs.sub_(coeffs.grad * lr)
    coeffs.grad.zero_()

In [ ]:
def train_one_epoch(coeffs: torch.Tensor, lr: float, train_features, train_target):
    loss = calc_loss(coeffs, train_features, train_target)
    loss.backward()
    with torch.no_grad():
        update_coeffs(coeffs, lr)
    print(f"Loss: {loss:.3f}")

In [ ]:
def train_model(epochs: int, lr: float, train_features, train_target):
    torch.manual_seed(42)
    coeffs = init_coeffs()
    for i in range(epochs):
        train_one_epoch(coeffs, lr, train_features, train_target)
    return coeffs

In [ ]:
def calc_acc(coeffs, features, target):
    return (target.bool() == (calc_preds(coeffs, features) > 0.5)).float().mean()

In [ ]:
train_split, val_split = RandomSplitter(seed=1337)(processed_df)

In [ ]:
train_features, val_features = t_features[train_split], t_features[train_split]
train_target, val_target = t_target[train_split], t_target[train_split]

In [ ]:
coeffs = train_model(
    epochs=18, lr=0.2, train_features=train_features, train_target=train_target
)

#### Getting predictions

In [ ]:
preds = calc_preds(coeffs, val_features)

In [ ]:
results = val_target.bool() == (preds > 0.5)

In [ ]:
results[:10]

Average accuracy

In [ ]:
results.float().mean()

Preds contain value smaller 0 and bigger 0 - use sigmoid function to get around that

In [ ]:
(preds < 0).any() and (preds > 1.0).any()

#### Train new model using sigmoid function to calculate predictions

In [ ]:
new_coeffs = train_model(
    epochs=30, lr=100, train_features=train_features, train_target=train_target
)

In [ ]:
calc_acc(coeffs=new_coeffs, features=val_features, target=val_target)

### Neural network

In [ ]:
def init_coeffs(n_hidden: int = 20):
    l1 = (torch.rand(t_features.shape[1], n_hidden) - 0.5) / n_hidden
    l2 = torch.randn(n_hidden, 1) - 0.3
    const = torch.rand(1)[0]
    return l1.requires_grad_(), l2.requires_grad_(), const.requires_grad_()

In [ ]:
def calc_preds(coeffs, features):
    l1, l2, const = coeffs
    result = F.relu(features @ l1)
    result = result @ l2 + const

    return torch.sigmoid(result)

In [ ]:
def update_coeffs(coeffs, lr):
    for layer in coeffs:
        layer.sub_(layer.grad * lr)
        layer.grad.zero_()

In [ ]:
coeffs_nn = train_model(
    epochs=18, lr=0.2, train_features=train_features, train_target=train_target
)

In [ ]:
calc_acc(coeffs=coeffs_nn, features=val_features, target=val_target)